In [2]:
!pip install pyarrow

     --------------------------------------- 20.6/20.6 MB 15.2 MB/s eta 0:00:00


In [3]:
import pyarrow.parquet as pq
import numpy as np
import pandas as pd
import pyarrow as pa

In [7]:
df = pd.read_parquet('Resources/flight_data_1.parquet')
pd.set_option('display.max_columns', None)
df.head()

,legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,isRefundable,isNonStop,baseFare,totalFare,seatsRemaining,totalTravelDistance,segmentsDepartureTimeEpochSeconds,segmentsDepartureTimeRaw,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
0,8b5148769053bda521a705606d3bab17,2022-09-28,2022-10-05,SFO,ORD,WAA0JXDS,PT7H5M,0,False,False,False,398.14,451.60,9,1933,1664992200||1665004800,2022-10-05T10:50:00.000-07:00||2022-10-05T14:2...,1664998080||1665017700,2022-10-05T12:28:00.000-07:00||2022-10-05T19:5...,LAS||ORD,SFO||LAS,United||United,UA||UA,Boeing 737-900||Boeing 777,5880||12900,424||1509,coach||coach
1,5c7220be73af699f9dab576f1a2fc158,2022-09-28,2022-10-05,SFO,ORD,WAA0JXDS,PT7H52M,0,False,False,False,398.14,451.60,2,1933,1665001500||1665016980,2022-10-05T13:25:00.000-07:00||2022-10-05T17:4...,1665007620||1665029820,2022-10-05T15:07:00.000-07:00||2022-10-05T23:1...,LAS||ORD,SFO||LAS,United||United,UA||UA,Boeing 737-800||Boeing 737-900,6120||12840,424||1509,coach||coach
2,a2bf9af0519df082262b84472a62ef61,2022-09-28,2022-10-05,SFO,ORD,WAA0JXDS,PT8H34M,1,False,False,False,398.14,451.60,9,1933,1665021060||1665039420,2022-10-05T18:51:00.000-07:00||2022-10-05T23:5...,1665026940||1665051900,2022-10-05T20:29:00.000-07:00||2022-10-06T05:2...,LAS||ORD,SFO||LAS,United||United,UA||UA,Boeing 737-800||Boeing 737-900,5880||12480,424||1509,coach||coach
3,eed542f7c1e5f60bf829d392782b922f,2022-09-28,2022-10-05,SFO,PHL,G03PXP4,PT7H49M,1,False,False,False,74.78,103.99,4,2590,1665016260||1665027600,2022-10-05T17:31:00.000-07:00||2022-10-05T20:4...,1665022260||1665044400,2022-10-05T19:11:00.000-07:00||2022-10-06T04:2...,LAS||PHL,SFO||LAS,Frontier Airlines||Frontier Airlines,F9||F9,||,6000||16800,424||2166,coach||coach
4,50e1dfeaecd825f72ec792880c580ed7,2022-09-28,2022-10-05,SFO,PHL,G03PXP4,PT13H19M,1,False,False,False,74.78,109.59,4,2590,1664996460||1665027600,2022-10-05T12:01:00.000-07:00||2022-10-05T20:4...,1665002580||1665044400,2022-10-05T13:43:00.000-07:00||2022-10-06T04:2...,LAS||PHL,SFO||LAS,Frontier Airlines||Frontier Airlines,F9||F9,||,6120||16800,424||2166,coach||coach


In [8]:
# Make boolean data numeric.
df["isBasicEconomy"] = df["isBasicEconomy"].replace("True", 1).replace("False", 0)
df["isRefundable"] = df["isRefundable"].replace("True", 1).replace("False", 0)
df["isNonStop"] = df["isNonStop"].replace("True", 1).replace("False", 0)
df.head()

,legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,isRefundable,isNonStop,baseFare,totalFare,seatsRemaining,totalTravelDistance,segmentsDepartureTimeEpochSeconds,segmentsDepartureTimeRaw,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
0,8b5148769053bda521a705606d3bab17,2022-09-28,2022-10-05,SFO,ORD,WAA0JXDS,PT7H5M,0,0,0,0,398.14,451.60,9,1933,1664992200||1665004800,2022-10-05T10:50:00.000-07:00||2022-10-05T14:2...,1664998080||1665017700,2022-10-05T12:28:00.000-07:00||2022-10-05T19:5...,LAS||ORD,SFO||LAS,United||United,UA||UA,Boeing 737-900||Boeing 777,5880||12900,424||1509,coach||coach
1,5c7220be73af699f9dab576f1a2fc158,2022-09-28,2022-10-05,SFO,ORD,WAA0JXDS,PT7H52M,0,0,0,0,398.14,451.60,2,1933,1665001500||1665016980,2022-10-05T13:25:00.000-07:00||2022-10-05T17:4...,1665007620||1665029820,2022-10-05T15:07:00.000-07:00||2022-10-05T23:1...,LAS||ORD,SFO||LAS,United||United,UA||UA,Boeing 737-800||Boeing 737-900,6120||12840,424||1509,coach||coach
2,a2bf9af0519df082262b84472a62ef61,2022-09-28,2022-10-05,SFO,ORD,WAA0JXDS,PT8H34M,1,0,0,0,398.14,451.60,9,1933,1665021060||1665039420,2022-10-05T18:51:00.000-07:00||2022-10-05T23:5...,1665026940||1665051900,2022-10-05T20:29:00.000-07:00||2022-10-06T05:2...,LAS||ORD,SFO||LAS,United||United,UA||UA,Boeing 737-800||Boeing 737-900,5880||12480,424||1509,coach||coach
3,eed542f7c1e5f60bf829d392782b922f,2022-09-28,2022-10-05,SFO,PHL,G03PXP4,PT7H49M,1,0,0,0,74.78,103.99,4,2590,1665016260||1665027600,2022-10-05T17:31:00.000-07:00||2022-10-05T20:4...,1665022260||1665044400,2022-10-05T19:11:00.000-07:00||2022-10-06T04:2...,LAS||PHL,SFO||LAS,Frontier Airlines||Frontier Airlines,F9||F9,||,6000||16800,424||2166,coach||coach
4,50e1dfeaecd825f72ec792880c580ed7,2022-09-28,2022-10-05,SFO,PHL,G03PXP4,PT13H19M,1,0,0,0,74.78,109.59,4,2590,1664996460||1665027600,2022-10-05T12:01:00.000-07:00||2022-10-05T20:4...,1665002580||1665044400,2022-10-05T13:43:00.000-07:00||2022-10-06T04:2...,LAS||PHL,SFO||LAS,Frontier Airlines||Frontier Airlines,F9||F9,||,6120||16800,424||2166,coach||coach


In [22]:
# View flight dates.
df["flightDate"]

0         2022-10-05
1         2022-10-05
2         2022-10-05
3         2022-10-05
4         2022-10-05
             ...    
351045    2022-11-09
351046    2022-11-09
351047    2022-11-09
351048    2022-11-09
351049    2022-11-09
Name: flightDate, Length: 351050, dtype: object

In [15]:
# Add new column.
df["daysBeforeFlight"] = pd.to_datetime(df["flightDate"]) - pd.to_datetime(df["searchDate"])
df.head()

,legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,isRefundable,isNonStop,baseFare,totalFare,seatsRemaining,totalTravelDistance,segmentsDepartureTimeEpochSeconds,segmentsDepartureTimeRaw,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode,daysBeforeFlight
0,8b5148769053bda521a705606d3bab17,2022-09-28,2022-10-05,SFO,ORD,WAA0JXDS,PT7H5M,0,0,0,0,398.14,451.60,9,1933,1664992200||1665004800,2022-10-05T10:50:00.000-07:00||2022-10-05T14:2...,1664998080||1665017700,2022-10-05T12:28:00.000-07:00||2022-10-05T19:5...,LAS||ORD,SFO||LAS,United||United,UA||UA,Boeing 737-900||Boeing 777,5880||12900,424||1509,coach||coach,7 days
1,5c7220be73af699f9dab576f1a2fc158,2022-09-28,2022-10-05,SFO,ORD,WAA0JXDS,PT7H52M,0,0,0,0,398.14,451.60,2,1933,1665001500||1665016980,2022-10-05T13:25:00.000-07:00||2022-10-05T17:4...,1665007620||1665029820,2022-10-05T15:07:00.000-07:00||2022-10-05T23:1...,LAS||ORD,SFO||LAS,United||United,UA||UA,Boeing 737-800||Boeing 737-900,6120||12840,424||1509,coach||coach,7 days
2,a2bf9af0519df082262b84472a62ef61,2022-09-28,2022-10-05,SFO,ORD,WAA0JXDS,PT8H34M,1,0,0,0,398.14,451.60,9,1933,1665021060||1665039420,2022-10-05T18:51:00.000-07:00||2022-10-05T23:5...,1665026940||1665051900,2022-10-05T20:29:00.000-07:00||2022-10-06T05:2...,LAS||ORD,SFO||LAS,United||United,UA||UA,Boeing 737-800||Boeing 737-900,5880||12480,424||1509,coach||coach,7 days
3,eed542f7c1e5f60bf829d392782b922f,2022-09-28,2022-10-05,SFO,PHL,G03PXP4,PT7H49M,1,0,0,0,74.78,103.99,4,2590,1665016260||1665027600,2022-10-05T17:31:00.000-07:00||2022-10-05T20:4...,1665022260||1665044400,2022-10-05T19:11:00.000-07:00||2022-10-06T04:2...,LAS||PHL,SFO||LAS,Frontier Airlines||Frontier Airlines,F9||F9,||,6000||16800,424||2166,coach||coach,7 days
4,50e1dfeaecd825f72ec792880c580ed7,2022-09-28,2022-10-05,SFO,PHL,G03PXP4,PT13H19M,1,0,0,0,74.78,109.59,4,2590,1664996460||1665027600,2022-10-05T12:01:00.000-07:00||2022-10-05T20:4...,1665002580||1665044400,2022-10-05T13:43:00.000-07:00||2022-10-06T04:2...,LAS||PHL,SFO||LAS,Frontier Airlines||Frontier Airlines,F9||F9,||,6120||16800,424||2166,coach||coach,7 days


In [24]:
# It seems that the search date is always 2022-09-28.
# Select all rows with a flight date other than 2022-10-05 in order to check daysBeforeFlight.
testdf = df
testdf = testdf.loc[testdf["flightDate"] != "2022-10-05"]
testdf.head()

,legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,isRefundable,isNonStop,baseFare,totalFare,seatsRemaining,totalTravelDistance,segmentsDepartureTimeEpochSeconds,segmentsDepartureTimeRaw,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode,daysBeforeFlight
48,f8d0e442cfeec534f1631956901c5f73,2022-09-28,2022-10-06,ATL,BOS,W07PXP5,PT9H44M,1,0,0,0,48.73,81.59,4,1518,1665111120||1665135300,2022-10-06T22:52:00.000-04:00||2022-10-07T05:3...,1665117060||1665146160,2022-10-07T00:31:00.000-04:00||2022-10-07T08:3...,MCO||BOS,ATL||MCO,Frontier Airlines||Frontier Airlines,F9||F9,||Airbus A321,5940||10860,399||1119,coach||coach,8 days
49,debcfe86332dc9772d19e314b30ed1a6,2022-09-28,2022-10-06,ATL,BOS,VA3NR,PT9H38M,0,0,0,0,74.00,181.58,0,None,1665078900||1665102000,2022-10-06T13:55:00.000-04:00||2022-10-06T20:2...,1665085500||1665113580,2022-10-06T15:45:00.000-04:00||2022-10-06T23:3...,FLL||BOS,ATL||FLL,Spirit Airlines||Spirit Airlines,NK||NK,||,6600||11580,None||None,coach||coach,8 days
50,4ae63b6ac5fa0c791ced0618842f5c37,2022-09-28,2022-10-06,ATL,BOS,N7AJZNB3,PT4H53M,0,1,0,0,152.56,186.10,7,956,1665060720||1665070200,2022-10-06T08:52:00.000-04:00||2022-10-06T11:3...,1665065580||1665078300,2022-10-06T10:13:00.000-04:00||2022-10-06T13:4...,CLT||BOS,ATL||CLT,American Airlines||American Airlines,AA||AA,Airbus A319||Airbus A321,4860||8100,228||728,coach||coach,8 days
51,fbaf00f6dcb766f4c52b29b3c0af3e05,2022-09-28,2022-10-06,ATL,BOS,N7AJZNB3,PT5H27M,0,1,0,0,152.56,186.10,7,956,1665054000||1665066060,2022-10-06T07:00:00.000-04:00||2022-10-06T10:2...,1665058620||1665073620,2022-10-06T08:17:00.000-04:00||2022-10-06T12:2...,CLT||BOS,ATL||CLT,American Airlines||American Airlines,AA||AA,Airbus A319||Airbus A321,4620||7560,228||728,coach||coach,8 days
52,6959a70f416c9186063144b4d5a69826,2022-09-28,2022-10-06,ATL,BOS,N7AJZNB3,PT6H11M,0,1,0,0,152.56,186.10,7,956,1665068940||1665083700,2022-10-06T11:09:00.000-04:00||2022-10-06T15:1...,1665073500||1665091200,2022-10-06T12:25:00.000-04:00||2022-10-06T17:2...,CLT||BOS,ATL||CLT,American Airlines||American Airlines,AA||AA,Airbus A319||Airbus A321,4560||7500,228||728,coach||coach,8 days


In [26]:
# Export table to csv.
df.to_csv('Resources/flight_data_clean_1.csv')

In [25]:
# Check file names.
for tableNumber in range(1, 6):
    print(f'Resources/flight_data_{tableNumber}.parquet')

Resources/flight_data_1.parquet
Resources/flight_data_2.parquet
Resources/flight_data_3.parquet
Resources/flight_data_4.parquet
Resources/flight_data_5.parquet


In [27]:
# Loop through the tables.
for tableNumber in range(1, 6):
    # Assign the path to a variable to use when accessing the file.
    table_import_path = f'Resources/flight_data_{tableNumber}.parquet'
    # Load the data frame.
    df = pd.read_parquet(table_import_path)
    # Make boolean data numeric.
    df["isBasicEconomy"] = df["isBasicEconomy"].replace("True", 1).replace("False", 0)
    df["isRefundable"] = df["isRefundable"].replace("True", 1).replace("False", 0)
    df["isNonStop"] = df["isNonStop"].replace("True", 1).replace("False", 0)
    # Add new column.
    df["daysBeforeFlight"] = pd.to_datetime(df["flightDate"]) - pd.to_datetime(df["searchDate"])
    # Export table to csv.
    table_export_path = f'Resources/flight_data_clean_{tableNumber}.csv'
    df.to_csv(table_export_path)